### Utilisation de Google Colab

Si vous utilisez Google Colab, suivez les instructions ci-dessous.

Tout d'abord, sélectionnez l'option GPU de Colab avec *Edit > Notebook settings* et sélectionner GPU comme Hardware accelerator. Installer ensuite deeplib avec la commande suivante:

In [ ]:
!pip install git+https://github.com/ulaval-damas/glo4030-labs.git

# Laboratoire 8: Transformers

Introduction aux transformers:
- Révolution dans le monde du NLP (Vaswani, 2017), maintenant les majeurs architectures de NLP sont toutes basées sur les transformers (BERT, GPT). Ex. Generative Pre-trained Transformer (GPT) et Bidirectional Encoder Representations from Transformers (BERT). ChatGPT basé sur GPT.
- Maintenant, les transformers sont aussi utilisés dans d'autres domaines du deep learning grâce à leur grande flexibilité et polyvalence. Vision par ordinateur: ViT, Swin, DETR, etc., Point Clouds: ..., Audio: ...
- Intéressant que le transformer de 2017 est encore utilisé! Seulement quelques petites modifications
- Une sorte d'architecture universelle qui peut traiter tout type de données
- Présentent une affinité pour le prétraining en raison de leur meilleur scaling avec la quantité de données et la taille du modèle (scaling vision transformer et Scaling laws for neural language models)

Transformers:
- Les transformers sont basés sur le principle d'attention
- Modèle ensemble -> ensemble de vecteurs (ensemble dans le sens mathématique)
- Réutilise des concepts intéressants: Connexion résiduelle (flot du gradient), LayerNorm, MLP
- Liens avec le message passing / algorithm


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np

import torch
import torch.random
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Partie 1: Scaled Dot-Product Attention
- Au coeur des transformers se trouve le principle d'attention
- Focus sur certaines parties de l'information, et moins sur d'autres
- Dans le cas des transformers, on utilise le "Scaled Dot-Product Attention" de Attention Is All You Need" (Vaswani, 2017)


### Champ réceptif

### Attention comme du passage de messages
- Graphe
- Q, K, aggregate V
- Graphe ont pas de positions -> fonction set de vecteurs vers set de vecteurs , doit encoder la position

### Self-attention vs cross-attention

### Le "Scaled" dans Scaled Dot-Product Attention
- Si k et v ~ N(0, 1)
- q @ k.T -> variance de d
- scaled par 1 / sqrt(d)
- empeche softmax de converger vers one-hot, aggrege information de seulement un token, plutot que plusieurs

In [11]:
batch_size = 4
num_tokens = 16
dimension = 64

q = torch.randn(batch_size, num_tokens, dimension)
k = torch.randn(batch_size, num_tokens, dimension)
attention = q @ k.transpose(-2, -1)
scaled_attention = attention * dimension ** -0.5

print(f'{k.var()}')
print(f'{v.var()}')
print(f'{attention.var()}')
print(f'{scaled_attention.var()}')

0.9649292230606079
0.9273311495780945
63.43318557739258
0.991143524646759


In [15]:
attention = torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)
scaled_attention = torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]) * np.sqrt(dimension), dim=-1)

print(attention)
print(scaled_attention)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])
tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])


## Partie 2: Multi-Head Attention
- Plusieurs têtes d'attention
- Plusieurs cannaux de communication séparés entre les noeuds

## Partie 3: TransformerEncoderLayer
- Multi-head self-attention
- Add and norm
- FF par token (attention = communication, FF est le token qui "pense")
- Connexion residuelle / skip (flot du gradient car l'addition distribue le grad egalement à chaque branche, aide à être deep, apprendre un delta)
- Prenorm vs postnorm

### BatchNorm vs LayerNorm
- LayerNorm ne demande pas de moyenne exponentielle, pas de difference entre train et test

In [58]:
torch.manual_seed(42)
batch_size = 32
dimension = 100
x = torch.randn(batch_size, dimension)
net = nn.Sequential(nn.Linear(100, 256), nn.ReLU(), nn.Linear(256, 256), nn.ReLU(), nn.Linear(256, 100))
x = net(x)

# BatchNorm (normalement utiliserait une moyenne exponentielle)
xmean_batch = x.mean(0, keepdim=True)
xvar_batch = x.var(0, keepdim=True)
x_batch = (x - xmean_batch) / torch.sqrt(xvar_batch + 1e-12)
print('BatchNorm')
print(f'Par batch:\n\tmean: {x_batch[:, 0].mean()}\n\tvar: {x_batch[:, 0].var()}')
print(f'Par layer:\n\tmean: {x_batch[0, :].mean()}\n\tvar: {x_batch[0, :].var()}')

# LayerNorm
xmean_layer = x.mean(-1, keepdim=True)
xvar_layer = x.var(-1, keepdim=True)
x_layer = (x - xmean_layer) / torch.sqrt(xvar_layer + 1e-12)
print('LayerNorm')
print(f'Par batch:\n\tmean: {x_layer[:, 0].mean()}\n\tvar: {x_layer[:, 0].var()}')
print(f'Par layer:\n\tmean: {x_layer[0, :].mean()}\n\tvar: {x_layer[0, :].var()}')

BatchNorm
Par batch:
	mean: -1.4901161193847656e-08
	var: 1.0
Par layer:
	mean: 0.026868706569075584
	var: 1.249971628189087
LayerNorm
Par batch:
	mean: 1.2997647523880005
	var: 0.41209426522254944
Par layer:
	mean: 2.38418573772492e-09
	var: 1.0


## Partie 4: TransformerEncoder
- Plusieurs couches de TransformerEncoderLayer

## Partie 5: Entraînement

## Partie 6: Vision Transformer
- Utilise PyTorch car plus efficace
- https://towardsdatascience.com/a-demonstration-of-using-vision-transformers-in-pytorch-mnist-handwritten-digit-recognition-407eafbc15b0